In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [19]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [10]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [20]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')


In [12]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


# Data Prep

Check if the missing values are presented in the features.
If there are missing values:
For categorical features, replace them with 'NA'
For numerical features, replace with with 0.0

In [5]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [9]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [15]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

In [22]:
df[categorical] = df[categorical].fillna('NA')

In [24]:
df[numerical] = df[numerical].fillna('0.0')

In [25]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [26]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

# Q1.

What is the most frequent observation (mode) for the column industry?


In [27]:
df['industry'].describe()

count       1462
unique         8
top       retail
freq         203
Name: industry, dtype: object

A: retail

# Q2.

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

interaction_count and lead_score
number_of_courses_viewed and lead_score
number_of_courses_viewed and interaction_count
annual_income and interaction_count
Only consider the pairs above when answering this question.


In [29]:
df[numerical].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


interaction_count and lead_score: 0.009888
number_of_courses_viewed and lead_score: -0.004879
number_of_courses_viewed and interaction_count: -0.023565
* annual_income and interaction_count: 0.027036

A: annual_income and interaction_count

# Split the data

Split your data in train/val/test sets with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
Make sure that the target value converted is not in your dataframe.

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 42)

In [32]:
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 42)

In [33]:
df_train = df_train.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)

In [36]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

# Q3. 

Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.
Round the scores to 2 decimals using round(score, 2).

Which of these variables has the biggest mutual information score?

industry
location
lead_source
employment_status

In [37]:
from sklearn.metrics import mutual_info_score

In [41]:
def mutual_info_converted_score(series):
    return mutual_info_score(series, df_full_train.converted)

In [45]:
mi =df_full_train[categorical].apply(mutual_info_converted_score)
mi.sort_values(ascending = False).round(2)

lead_source          0.03
employment_status    0.01
industry             0.01
location             0.00
dtype: float64

A: lead_source 

# Q4.

Now let's train a logistic regression.
Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
Fit the model on the training dataset.
To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
What accuracy did you get?

0.64
0.74
0.84
0.94


In [78]:
# Vectorize the datasets

from sklearn.feature_extraction import DictVectorizer

In [120]:
dicts_train = df_train[categorical+numerical].to_dict(orient = "records")

In [80]:
dv = DictVectorizer(sparse = False)

In [121]:
X_train = dv.fit_transform(dicts_train)

In [122]:
val_dict = df_val[categorical+numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [83]:
# Fit into Logic regression model

from sklearn.linear_model import LogisticRegression


In [84]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [123]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [124]:
# Make prediction

y_pred = model.predict_proba(X_val)[:,1]

In [125]:
converted_decision = y_pred >= 0.5

In [128]:
# Accuracy of our prediction

score_orig = (y_val == converted_decision).mean()
score_orig

0.6996587030716723

The mean accuracy of the prediction is only 70%.

# Q5.

Let's find the least useful feature using the feature elimination technique.
Train a model using the same features and parameters as in Q4 (without rounding).
Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
Which of following feature has the smallest difference?

'industry'
'employment_status'
'lead_score'

## no industry

In [111]:
df_train_no_industry = df_train.drop('industry', axis=1)

In [112]:
dicts_train_no_industry = df_train_no_industry.to_dict(orient = 'records')

In [113]:
X_train_no_industry = dv.fit_transform(dicts_train_no_industry)

In [114]:
dicts_val_no_industry = df_val.drop('industry',axis=1).to_dict(orient='records')
X_val_no_industry = dv.transform(dicts_val_no_industry)

In [115]:
model.fit(X_train_no_industry, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [116]:
y_pred = model.predict_proba(X_val_no_industry)[:,1]

In [117]:
converted_decision = y_pred >= 0.5

In [119]:
score_industry = (y_val == converted_decision).mean()

In [130]:
score_industry

0.6996587030716723

In [132]:
score_orig - score_industry

0.0

## no employment_status

In [133]:
df_train_no_employment_status = df_train.drop('employment_status', axis=1)

In [134]:
dicts_train_no_employment_status = df_train_no_employment_status.to_dict(orient = 'records')

In [135]:
X_train_no_employment_status = dv.fit_transform(dicts_train_no_employment_status)

In [136]:
dicts_val_no_employment_status = df_val.drop('employment_status',axis=1).to_dict(orient='records')
X_val_no_employment_status = dv.transform(dicts_val_no_employment_status)

In [137]:
model.fit(X_train_no_employment_status, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [138]:
y_pred = model.predict_proba(X_val_no_employment_status)[:,1]

In [139]:
converted_decision = y_pred >= 0.5

In [141]:
score_emp = (y_val == converted_decision).mean()

In [142]:
score_orig - score_emp

0.0034129692832763903

## no lead_score

In [145]:
df_train_no_lead_score = df_train.drop('lead_score', axis=1)

In [146]:
dicts_train_no_lead_score = df_train_no_lead_score.to_dict(orient = 'records')

In [147]:
X_train_no_lead_score = dv.fit_transform(dicts_train_no_lead_score)

In [148]:
dicts_val_no_lead_score = df_val.drop('lead_score',axis=1).to_dict(orient='records')
X_val_no_lead_score = dv.transform(dicts_val_no_lead_score)

In [149]:
model.fit(X_train_no_lead_score, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [150]:
y_pred = model.predict_proba(X_val_no_lead_score)[:,1]

In [151]:
converted_decision = y_pred >= 0.5

In [152]:
score_lead = (y_val == converted_decision).mean()

In [153]:
score_orig - score_lead

-0.0034129692832765013

In [ ]:
A: 'lead_score'

# Q6.

Now let's train a regularized logistic regression.
Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
Train models using all the features as in Q4.
Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
Which of these C leads to the best accuracy on the validation set?

0.01
0.1
1
10
100

In [156]:
parameters = [0.01, 0.1, 1, 10, 100]

# vectorize train
dicts_train = df_train[categorical+numerical].to_dict(orient = 'records') 
dicts_val = df_val[categorical+numerical].to_dict(orient = 'records')

dv = DictVectorizer(sparse=False)

X_train = dv.fit_transform(dicts_train)
X_val = dv.transform(dicts_val)
    
for c in parameters:
    #fit into model
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42) 
    model.fit(X_train, y_train)
    
    #validate the model
    y_pred = model.predict_proba(X_val)[:,1]
    converted_decision = y_pred >= 0.5
    score = (y_val == converted_decision).mean()
    print(c, score)

0.01 0.6996587030716723
0.1 0.6996587030716723
1 0.6996587030716723
10 0.6996587030716723
100 0.6996587030716723
